# TUTORIAL PYTORCH
## Redes Neuronales
###### Fuente: [Documentación Oficial de Pytorch](https://pytorch.org/tutorials/)
###### Edición y traducción por Cristobal Donoso O. 
###### Agosto 2018

Para construir redes neuronales podemos utilizar el paquete ```torch.nn```. Logicamente, este paquete depende de la clase Autograd; Si aún no haz visto el notebook de autograd [puedes verlo aquí](Autograd.ipynb).<br>La idea consiste en crear modelos mediante la construcción de un **grafo de operaciones** y luego **calcular automaticamente sus gradientes** para ajustar los pesos.

Considere el siguiente modelo:
<img src="https://pytorch.org/tutorials/_images/mnist.png">

Esto corresponde al feed-forward de una red [neuronal convolucional](https://www.deeplearningbook.org/contents/convnets.html). Basicamente, mapea cada pixel desde la entrada ```(32x32)``` y realiza convoluciones con el objetivo de extraer caracteristicas más importantes para la clasificación. Finalmente, los datos pasan por tres capas fullyconnected de ```120``` , ```84``` y ```10 neuronas``` que realizarán la clasificación mediante combinaciones lineales y alguna función de activación.<br><br>
>En general para entrenar una red neuronal necesitamos:
- Definir la red neuronal cuya estructura tiene parametros ajustables (por ej. pesos y bias)
- Iterar sobre un dataset de entrada
- Procesar la entrada atraves de la red (forward)
- Computar la perdida asociada a la clasificación (en aprendizaje supervisado tenemos etiquetas)
- Retropropagar el error usando los gradientes
- Actualizar los peoss de la red, tipicamente usando: 
    - ```weight = weight - learning_rate * gradient ```

#### Definiendo la red

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

La clase ```nn.Module``` contiene funciones basicas para cualquier red neuronal. Permite ademas interactuar con la GPU exportando e importando valores. Todo modelo debe heredar de esta clase

In [27]:
class Net(nn.Module):
    #Constructor
    def __init__(self):
        # Primero definimos las variables que conformarán nuestra red.
        # Note que éstas son independientes entre ellas - Simplemente estamos declarando.
        super(Net, self).__init__()
        # 1 canal para la imagen de entrada, 
        # 6 canales para el output, 
        # 5x5 convolución cuadrada (basta con colocar solo un numeor)
    
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Operacione slineales del tipo: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling sobre una ventana de (2, 2)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Note que si el tamaño es cuadrado (2,2) = 2. Por lo tanto basta con colocar solo un número
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # Hacemos una flat para generar la entrada a las fullyconnected
        x = x.view(-1, self.num_flat_features(x))
        #La salida de cada combinación lineal pasa por una funcion de activación ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # La última capa no tiene función de activación
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # todas las dimensiones con excepeción de la del batch (0)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Solo tienes que definir la función ```forward``` ya que ```backward``` está definida en la clase ```Autograd```. Puedes utilizar cualquier Tensor definido anteriormente.<br>Para ver los parametros de un modelo podemos ejecutar ```net.parameters()```

In [28]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Los pesos de la primera capa convolucional

10
torch.Size([6, 1, 5, 5])


Ahora vamos a generar una entrada aleatoria de ```32x32```

In [29]:
input = torch.randn(1, 1, 32, 32)
out = net.forward(input) #Alternativamente puedes colocar net(input) y automaticamente buscará el método forward
print(out)

tensor([[-0.0192, -0.0636,  0.0127,  0.0928,  0.0044,  0.0761,  0.0456,  0.0443,
          0.0131, -0.0162]], grad_fn=<ThAddmmBackward>)


Inicializamos los buffers de todos los gradientes con cero y luego retropropagamos con gradientes aleatorios

In [30]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### Calculando la perdida
Puedes encontrar funciones de perdida [aquí](https://pytorch.org/docs/stable/nn.html). Una función tipica para computar la perdida es el **Error Medio Cuadrático** (del ingles MSE) y se encuentra en ```nn.MSELoss```. Esta función compara la salida con una etiqueta real 

In [72]:
output = net(input)
target = torch.randn(10)  # etiquetas reales aleatorias
target = target.view(1, -1)  # modificamos sus dimensiones para que sea de la misma forma que la salida 
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5347, grad_fn=<MseLossBackward>)


Ahora si seguimos la perdida en la dirección del backward, usando el atributo ```.grad_fn```, podras ver un grado de operaciones como este:
```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu 
      -> linear -> relu -> linear
      -> MSELoss
      -> loss
```
Podemos imprimir la estructura del grafo haciendo uso de las Funciones de cada Tensor

In [76]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0])  # Linear

Ahora si ejecutamos ```loss.backward()``` el grafo completo será diferenciado respecto a la perdida y todos los Tensores del grafo que tengan ```requires_grad=True``` tendrán su gradiente almacenado en su atributo ```.grad```

In [77]:
net.zero_grad()     # Dejamos en 0 todos los buffers para los gradientes de los parametros

print('conv1.bias.grad antes del backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad despues del backward')
print(net.conv1.bias.grad)

conv1.bias.grad antes del backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad despues del backward
tensor([ 0.0005,  0.0149,  0.0007, -0.0224, -0.0040,  0.0020])


#### Actualizando los pesos
Una vez que calculamos los gradientes derivando con respecto a la perdida, debemos actualizar los pesos. Utilizando la formula del Gradiente Descendente Estocástico (SGD)

    weight = weight - learning_rate * gradient

Podriamos computar esto con un ciclo for simple

In [78]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

Sin embargo, podemos utilizar distintos optimizadores que ya se encuentran implementados en ```torch.optim```

In [79]:
import torch.optim as optim
# Creamos el Optimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

Realizaremos el proceso de entrenamiento nuevamente 

In [81]:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
# Para hacer la actualización de pesos 
optimizer.step()